<a href="https://colab.research.google.com/github/AnkitaChakraborty1/Capstone_Adversarial_Robustness/blob/main/SearchMethod_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
 pip install transformers

In [10]:
!pip3 install textattack[tensorflow]

In [11]:
pip install keras

##Importing all the important libraries

In [12]:
from textattack import Attack
from textattack.constraints.pre_transformation import (
            RepeatModification,
                StopwordModification,
                )
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import GreedyWordSwapWIR,AlzantotGeneticAlgorithm,BeamSearch,ImprovedGeneticAlgorithm,ParticleSwarmOptimization,GreedySearch
from textattack.transformations import WordSwapQWERTY
from textattack.transformations import (
    CompositeTransformation,
    WordSwapNeighboringCharacterSwap,
    WordSwapQWERTY,
    WordSwapWordNet,
    WordSwapRandomCharacterDeletion,
    WordSwapRandomCharacterInsertion,
    WordSwapRandomCharacterSubstitution,
)
from textattack.constraints.overlap import LevenshteinEditDistance
#from textattack.attack_recipe import AttackRecipe

from abc import ABC, abstractmethod

from textattack import Attack
from itertools import combinations
import transformers
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack import Attack
from textattack.search_methods import GreedySearch
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.goal_functions import UntargetedClassification
from textattack.transformations import WordSwapEmbedding
from textattack.constraints.pre_transformation import RepeatModification
from textattack.constraints.pre_transformation import StopwordModification
from textattack import AttackArgs
from textattack.datasets import Dataset
from textattack.datasets import HuggingFaceDataset
from textattack import Attacker
from textattack.loggers import CSVLogger # tracks a dataframe for us.



## Creating the Keras Model

In [13]:
import tensorflow as tf
import tensorflow.keras
import numpy as np
from tensorflow.keras.utils import to_categorical
from textattack.models.wrappers import ModelWrapper
from textattack.datasets import HuggingFaceDataset
from textattack.attack_recipes import PWWSRen2019

import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout

from nltk.tokenize import word_tokenize, RegexpTokenizer

In [14]:

NUM_WORDS = 1000

(x_train_tokens, y_train), (x_test_tokens, y_test) = tf.keras.datasets.imdb.load_data(
    path="imdb.npz",
    num_words=NUM_WORDS,
    skip_top=0,
    maxlen=None,
    seed=113,
    start_char=1,
    oov_char=2,
    index_from=3
)

def transform(x):
  x_transform = []
  for i, word_indices in enumerate(x):
    BoW_array = np.zeros((NUM_WORDS,))
    for index in word_indices:
      if index < len(BoW_array):
        BoW_array[index] += 1
    x_transform.append(BoW_array)
  return np.array(x_transform)


index = int(0.9 * len(x_train_tokens))
x_train = transform(x_train_tokens)[:index]
x_test = transform(x_test_tokens)[index:]
y_train = np.array(y_train[:index])
y_test = np.array(y_test[index:])
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

vocabulary = tf.keras.datasets.imdb.get_word_index(
    path='imdb_word_index.json'
)

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specif

In [15]:
#Model Created with Keras
model = Sequential()
model.add(Dense(512, activation='relu', input_dim=NUM_WORDS))
model.add(Dropout(0.3))
model.add(Dense(100, activation='relu'))
model.add(Dense(2, activation='sigmoid'))
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.00001)

model.compile(
 optimizer = opt,
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)


results = model.fit(
 x_train, y_train,
 epochs= 18,
 batch_size = 512,
 validation_data = (x_test, y_test)
)


print(results.history)

Epoch 1/18
44/44 [==============================] - 1s 7ms/step - loss: 1.7233 - accuracy: 0.5001 - val_loss: 1.1907 - val_accuracy: 0.4912
Epoch 2/18
44/44 [==============================] - 0s 4ms/step - loss: 1.1413 - accuracy: 0.5058 - val_loss: 0.7659 - val_accuracy: 0.5240
Epoch 3/18
44/44 [==============================] - 0s 4ms/step - loss: 0.9452 - accuracy: 0.5131 - val_loss: 0.7066 - val_accuracy: 0.5388
Epoch 4/18
44/44 [==============================] - 0s 4ms/step - loss: 0.9034 - accuracy: 0.5324 - val_loss: 0.6947 - val_accuracy: 0.5672
Epoch 5/18
44/44 [==============================] - 0s 4ms/step - loss: 0.8703 - accuracy: 0.5438 - val_loss: 0.6823 - val_accuracy: 0.5900
Epoch 6/18
44/44 [==============================] - 0s 4ms/step - loss: 0.8568 - accuracy: 0.5482 - val_loss: 0.6705 - val_accuracy: 0.6088
Epoch 7/18
44/44 [==============================] - 0s 4ms/step - loss: 0.8370 - accuracy: 0.5653 - val_loss: 0.6594 - val_accuracy: 0.6228
Epoch 8/18
44/44 [==

In [16]:
class CustomKerasModelWrapper(ModelWrapper):
    def __init__(self, model):
        self.model = model

    def __call__(self, text_input_list):

      x_transform = []
      for i, review in enumerate(text_input_list):
        tokens = [x.strip(",") for x in review.split()]
        BoW_array = np.zeros((NUM_WORDS,))
        for word in tokens:
          if word in vocabulary:
            if vocabulary[word] < len(BoW_array):
              BoW_array[vocabulary[word]] += 1
        x_transform.append(BoW_array)
      x_transform = np.array(x_transform)
      prediction = self.model.predict(x_transform)
      return prediction


CustomKerasModelWrapper(model)(["bad bad bad bad bad", "good good good good"])

array([[0.4702987 , 0.54213136],
       [0.46177417, 0.5005922 ]], dtype=float32)

In [17]:
class Recipe(Attack,ABC):
    @staticmethod
    def build(model_wrapper,search_m):
        transformation = WordSwapRandomCharacterSubstitution()
                
        constraints = [RepeatModification(), StopwordModification()]
       
        constraints.append(LevenshteinEditDistance(30))
        
        goal_function = UntargetedClassification(model_wrapper)

        if(search_m == "BeamSearch"):
            search_method = BeamSearch(beam_width = 4)
        elif(search_m == "ImprovedGeneticAlgorithm"):
            search_method = ImprovedGeneticAlgorithm(pop_size=60,
                   max_iters=20, temp=0.3, give_up_if_no_improvement=False, post_crossover_check=True, max_crossover_retries=20, max_replace_times_per_index=5)
        elif(search_m == "ParticleSwarmOptimization"):
            search_method = ParticleSwarmOptimization(pop_size=60, max_iters=20, post_turn_check=True, max_turn_retries=20)
        elif(search_m == "GreedySearch"):
            search_method = GreedySearch()
        elif(search_m == "GreedyWordSwap"):
            search_method = GreedyWordSwapWIR("weighted-saliency")
        elif(search_m == "AlzantotGeneticAlgorithm"):
            search_method = AlzantotGeneticAlgorithm(pop_size = 60,
                   max_iters = 20, temp = 0.3, give_up_if_no_improvement = False, post_crossover_check = False, max_crossover_retries = 20)


        return Attack(goal_function, constraints, transformation, search_method)


## Results for search methods:  'BeamSearch','ImprovedGeneticAlgorithm','ParticleSwarmOptimization','GreedySearch','GreedyWordSwap','AlzantotGeneticAlgorithm'

Num_samples : 5

In [ ]:

search_method_list = ['BeamSearch','ImprovedGeneticAlgorithm','ParticleSwarmOptimization','GreedySearch','GreedyWordSwap','AlzantotGeneticAlgorithm']

dataset = HuggingFaceDataset("imdb", split="test")
for search_m in search_method_list:
#print("Starting attack for-----------------",recipe)
    attack = Recipe.build(model_wrapper,search_m)
    attack_args = AttackArgs(num_examples=5,log_to_csv="log.csv",checkpoint_interval=5,checkpoint_dir="checkpoints")
    attacker = Attacker(attack, dataset, attack_args)
    attack_results = attacker.attack_dataset()

Reusing dataset imdb (/root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3)
textattack: Loading datasets dataset imdb, split test.
textattack: Unknown if model of class <class 'tensorflow.python.keras.engine.sequential.Sequential'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path log.csv

  0%|          | 0/5 [00:00<?, ?it/s]

Attack(
  (search_method): BeamSearch(
    (beam_width):  4
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapRandomCharacterSubstitution(
    (random_one):  True
  )
  (constraints): 
    (0): LevenshteinEditDistance(
        (max_edit_distance):  30
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 




 20%|██        | 1/5 [00:10<00:41, 10.28s/it]
[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  20%|██        | 1/5 [00:10<00:41, 10.29s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[Positive (53%)]] --> [[Negative (50%)]]

I [[went]] and saw this movie last [[night]] after being coaxed to by a few friends of mine. I'll [[admit]] that I was reluctant to see it because from what I knew of Ashton Kutcher he was only able to do comedy. I was wrong. Kutcher [[played]] the character of Jake Fischer very [[well]], and Kevin Costner [[played]] Ben Randall with such professionalism. The sign of a good movie is that it can toy with our emotions. This one did exactly that. The entire theater (which was sold out) was overcome by laughter during the first half of the movie, and were moved to tears during the second half. While exiting the theater I not only [[saw]] many women in tears, but many full grown men as [[well]], trying desperately not to [[let]] anyone see them crying. This movie was great, and I suggest that you go see it before you judge.

I [[wcnt]] and saw this 


[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  40%|████      | 2/5 [01:10<01:45, 35.20s/it]
[Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:  40%|████      | 2/5 [01:10<01:45, 35.21s/it]
[Succeeded / Failed / Skipped / Total] 2 / 0 / 1 / 3:  60%|██████    | 3/5 [01:10<00:46, 23.49s/it]

...

--------------------------------------------- Result 4 ---------------------------------------------
[[Positive (51%)]] --> [[Negative (50%)]]

I [[saw]] this film in a sneak preview, and it is delightful. The cinematography is unusually creative, the [[acting]] is good, and the story is fabulous. If this movie does not do [[well]], it won't be because it doesn't deserve to. Before this film, I didn't realize how charming Shia Lebouf could be. [[He]] does a marvelous, self-contained, job as the lead. There's something [[incredibly]] sweet about him, and it makes the movie even better. The other actors do a good job as well, and the film contains [[moments]] of [[really]] [[high]] suspense, more than one might expect from a movie about golf. Sports movies are a dime a dozen, but this one stands out. <br /><br />This is one I'd recommend to anyone.

I [[Qaw]] this film in a sneak preview, and it is delightful. The cinematography is unusually creative, the [[actiCg]] is good, and the stor


[Succeeded / Failed / Skipped / Total] 3 / 0 / 1 / 4: 100%|██████████| 5/5 [06:29<00:00, 77.82s/it]
[Succeeded / Failed / Skipped / Total] 4 / 0 / 1 / 5: 100%|██████████| 5/5 [06:29<00:00, 77.82s/it]textattack: Saving checkpoint under "checkpoints/1645074963840.ta.chkpt" at 2022-02-17 05:16:03 after 5 attacks.


--------------------------------------------- Result 5 ---------------------------------------------
[[Positive (53%)]] --> [[Negative (51%)]]

Bill Paxton has taken the true story of the 1913 US golf [[open]] and made a film that is about much more than an extra-ordinary game of golf. The film also deals directly with the [[class]] tensions of the early twentieth century and touches upon the profound anti-Catholic prejudices of both the British and American establishments. But at heart the film is about that perennial favourite of triumph against the odds.<br /><br />The acting is exemplary throughout. Stephen Dillane is excellent as [[usual]], but the revelation of the movie is Shia LaBoeuf who delivers a disciplined, dignified and highly sympathetic performance as a [[working]] [[class]] Franco-Irish kid [[fighting]] his way through the prejudices of the New England WASP establishment. For those who are only familiar with his slap-stick [[performances]] in "Even Stevens" this demons

[Succeeded / Failed / Skipped / Total] 4 / 0 / 1 / 5: 100%|██████████| 5/5 [06:29<00:00, 77.82s/it]


+-------------------------------+---------+
| Attack Results                |         |
+-------------------------------+---------+
| Number of successful attacks: | 4       |
| Number of failed attacks:     | 0       |
| Number of skipped attacks:    | 1       |
| Original accuracy:            | 80.0%   |
| Accuracy under attack:        | 0.0%    |
| Attack success rate:          | 100.0%  |
| Average perturbed word %:     | 5.78%   |
| Average num. words per input: | 215.0   |
| Avg num queries:              | 2137.75 |
+-------------------------------+---------+


## Results for search methods:  'BeamSearch','ImprovedGeneticAlgorithm'

Num_samples : 10

In [ ]:
from textattack import AttackArgs
from textattack.datasets import Dataset
from textattack import Attacker

model_wrapper = CustomKerasModelWrapper(model)
search_method_list = ['BeamSearch','ImprovedGeneticAlgorithm']

dataset = HuggingFaceDataset("imdb", split="test")
for search_m in search_method_list:
    print("Starting attack for search method-----------------",search_m)
    attack = Recipe.build(model_wrapper,search_m)
    attack_args = AttackArgs(num_examples=10,log_to_csv=f"search_method_${search_m}.csv",checkpoint_interval=5,checkpoint_dir="checkpoints")
    attacker = Attacker(attack, dataset, attack_args)
    attack_results = attacker.attack_dataset()

textattack: Loading datasets dataset imdb, split test.
textattack: Unknown if model of class <class 'tensorflow.python.keras.engine.sequential.Sequential'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path search_method_$BeamSearch.csv
  0%|          | 0/10 [00:00<?, ?it/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3. Subsequent calls will reuse this data.
Starting attack for search method----------------- BeamSearch
Attack(
  (search_method): BeamSearch(
    (beam_width):  4
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapRandomCharacterSubstitution(
    (random_one):  True
  )
  (constraints): 
    (0): LevenshteinEditDistance(
        (max_edit_distance):  30
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 2 / 3:  30%|███       | 3/10 [00:05<00:13,  1.88s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[Positive (53%)]] --> [[Negative (50%)]]

I went and saw this movie last night after being coaxed to by a few friends of mine. I'll admit that I was reluctant to see it because from what I knew of Ashton Kutcher he was only able to do comedy. I was wrong. Kutcher [[played]] the character of Jake Fischer very well, and Kevin Costner [[played]] Ben Randall with such professionalism. The sign of a good movie is that it can toy with our emotions. This one did exactly that. The entire theater (which was sold out) was overcome by laughter during the first [[half]] of the movie, and were moved to tears during the second half. While exiting the theater I not only saw many women in tears, but many full grown men as well, trying desperately not to let [[anyone]] see them crying. This movie was great, and I suggest that you go see it before you judge.

I went and saw this movie last night after b

[Succeeded / Failed / Skipped / Total] 2 / 0 / 3 / 5:  50%|█████     | 5/10 [00:06<00:06,  1.22s/it]textattack: Saving checkpoint under "checkpoints/1645104663924.ta.chkpt" at 2022-02-17 13:31:03 after 5 attacks.


...

[Succeeded / Failed / Skipped / Total] 3 / 0 / 5 / 8:  80%|████████  | 8/10 [00:08<00:02,  1.06s/it]

--------------------------------------------- Result 8 ---------------------------------------------
[[Positive (51%)]] --> [[Negative (50%)]]

I felt this film did have many good qualities. The cinematography was certainly different exposing the stage aspect of the set and story. The [[original]] characters as actors was certainly an achievement and I felt most played quite convincingly, of course they are playing themselves, but definitely unique. The cultural aspects may leave many disappointed as a familiarity with the Chinese and Oriental culture will answer a lot of questions regarding parent/child relationships and the stigma that goes with any drug use. I found the Jia Hongsheng story interesting. On a down note, the story is in Beijing and some of the fashion and music reek of early 90s even though this was made in 2001, so it's really cheesy sometimes (the Beatles crap, etc). Whatever, not a top [[ten]] or twenty but if it's on the television, check it out.

I felt this film 

[Succeeded / Failed / Skipped / Total] 4 / 0 / 5 / 9:  90%|█████████ | 9/10 [00:15<00:01,  1.77s/it]

--------------------------------------------- Result 9 ---------------------------------------------
[[Positive (52%)]] --> [[Negative (50%)]]

This [[movie]] is amazing because the fact that the real people portray themselves and their real life experience and do such a good job it's like they're almost living the past over again. Jia Hongsheng plays himself an actor who quit everything except music and drugs struggling with depression and searching for the meaning of life while being angry at everyone especially the people who [[care]] for him most. There's [[moments]] in the [[movie]] that will make you wanna cry because the family especially the father did such a good job. However, this movie is not for everyone. Many people who suffer from depression will understand Hongsheng's problem and why he does the things he does for example keep himself shut in a dark room or go for walks or bike rides by himself. Others might see the movie as boring because it's just so real that its almo

[Succeeded / Failed / Skipped / Total] 4 / 0 / 5 / 9:  90%|█████████ | 9/10 [00:30<00:03,  3.36s/it]

##Results for search methods: 'ParticleSwarmOptimization','GreedySearch','GreedyWordSwap','AlzantotGeneticAlgorithm'

In [ ]:
from textattack import AttackArgs
from textattack.datasets import Dataset
from textattack import Attacker

model_wrapper = CustomKerasModelWrapper(model)
search_method_list = ['ParticleSwarmOptimization','GreedySearch','GreedyWordSwap','AlzantotGeneticAlgorithm']

dataset = HuggingFaceDataset("imdb", split="test")
for search_m in search_method_list:
    print("Starting attack for search method-----------------",search_m)
    attack = Recipe.build(model_wrapper,search_m)
    attack_args = AttackArgs(num_examples=10,log_to_csv=f"search_method_${search_m}.csv",checkpoint_interval=5,checkpoint_dir="checkpoints")
    attacker = Attacker(attack, dataset, attack_args)
    attack_results = attacker.attack_dataset()

textattack: Loading datasets dataset imdb, split test.
textattack: Unknown if model of class <class 'tensorflow.python.keras.engine.sequential.Sequential'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path search_method_$ParticleSwarmOptimization.csv
  0%|          | 0/10 [00:00<?, ?it/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3. Subsequent calls will reuse this data.
Starting attack for search method----------------- ParticleSwarmOptimization
Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  60
    (max_iters):  20
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapRandomCharacterSubstitution(
    (random_one):  True
  )
  (constraints): 
    (0): LevenshteinEditDistance(
        (max_edit_distance):  30
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 1 / 2:  20%|██        | 2/10 [00:03<00:14,  1.87s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[Positive (50%)]] --> [[Negative (50%)]]

I went and saw this movie last night after being coaxed to by a few [[friends]] of mine. I'll admit that I was reluctant to see it because from what I knew of Ashton Kutcher he was only able to do comedy. I was wrong. Kutcher played the character of Jake Fischer very well, and Kevin Costner played Ben Randall with such professionalism. The sign of a good movie is that it can toy with our emotions. This one did exactly that. The entire theater (which was sold out) was overcome by laughter during the first half of the movie, and were moved to tears during the second half. While exiting the theater I not only saw many women in tears, but many full grown men as well, trying desperately not to let anyone see them crying. This movie was great, and I suggest that you go see it before you judge.

I went and saw this movie last night after being coaxed 

[Succeeded / Failed / Skipped / Total] 2 / 0 / 2 / 4:  40%|████      | 4/10 [00:10<00:16,  2.69s/it]

...

--------------------------------------------- Result 6 ---------------------------------------------
[[Negative (52%)]] --> [[[SKIPPED]]]

I saw this film on September 1st, 2005 in Indianapolis. I am one of the judges for the Heartland Film Festival that screens films for their Truly Moving Picture Award. A Truly Moving Picture "...explores the human journey by artistically expressing hope and respect for the positive values of life." Heartland gave that award to this film.<br /><br />This is a story of golf in the early part of the 20th century. At that time, it was the game of upper class and rich "gentlemen", and working people could only participate by being caddies at country clubs. With this backdrop, this based-on-a-true-story unfolds with a young, working class boy who takes on the golf establishment and the greatest golfer in the world, Harry Vardon.<br /><br />And the story is inspirational. Against all odds, Francis Ouimet (played by Shia LaBeouf of "Holes") gets to compete 

[Succeeded / Failed / Skipped / Total] 4 / 0 / 6 / 10: 100%|██████████| 10/10 [02:14<00:00, 13.49s/it]textattack: Saving checkpoint under "checkpoints/1645096944849.ta.chkpt" at 2022-02-17 11:22:24 after 10 attacks.


--------------------------------------------- Result 7 ---------------------------------------------
[[Positive (52%)]] --> [[Negative (50%)]]

Maybe I'm reading into this too much, but I wonder how much of a hand Hongsheng had in developing the film. I mean, when a story is told casting the main character as himself, I would think he would be a heavy hand in writing, documenting, etc. and that would make it a little biased.<br /><br />But...his family and friends also [[may]] have had a hand in getting the actual details about Hongsheng's life. I think the [[best]] view would have been told from Hongsheng's family and friends' perspectives. They saw his transformation and weren't so messed up on drugs that they remember everything.<br /><br />As for Hongsheng being full of himself, the consistencies of the Jesus Christ pose [[make]] him [[appear]] as a martyr who sacrificed his life (metaphorically, of course, [[he's]] obviously still alive as he was cast as himself) for his family's 

[Succeeded / Failed / Skipped / Total] 4 / 0 / 6 / 10: 100%|██████████| 10/10 [02:14<00:00, 13.49s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 4      |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 6      |
| Original accuracy:            | 40.0%  |
| Accuracy under attack:        | 0.0%   |
| Attack success rate:          | 100.0% |
| Average perturbed word %:     | 1.18%  |
| Average num. words per input: | 299.0  |
| Avg num queries:              | 560.5  |
+-------------------------------+--------+